In [9]:
import json

In [10]:
with open("data_graph.json","r") as f:
    dict_p = json.load(f)

In [11]:
nodes = []
relations = []
for p in dict_p["paragraphs"]:
    nodes += p["graph"]["nodes"]
    relations += p["graph"]["relations"]

In [12]:
with open("hierarchy.json","r") as f:
  hierarchy_dict = json.load(f)

In [13]:
list_of_cypher_queries = []

def building_hierarchy_tree(parent,hierarchy_dict,list_of_cypher_queries):
    for title,content in hierarchy_dict.items():
        label = content["label"]
        list_of_cypher_queries += ['CREATE (:' + label.lower()+ ' {name: "'+ title +'"});']
        if parent is not None :
            list_of_cypher_queries += ['MATCH (a:'+parent['label'].lower()+'{name:"'+parent['name']+'"}), (b:' + label.lower() +' {name:"' + title +'"})' + '  CREATE (b)-[:IS_PART_OF]->(a);']
        if "childs" in content.keys():
            list_of_cypher_queries = building_hierarchy_tree({'name':title,'label':label},content["childs"],list_of_cypher_queries)
    return list_of_cypher_queries

list_of_cypher_queries = building_hierarchy_tree(None,hierarchy_dict,list_of_cypher_queries)

In [14]:

def build_node_name(title,number):
    if "Abstract" in title:
        return "Abstract P1"
    else:
        if title[2].isnumeric():
            if title[4].isnumeric():
                return title[:5]+ " P" + str(number)
            else:
                return title[:3] + " P" + str(number)
        else:
            return title[:2] + " P" + str(number)
        


In [7]:
for p in dict_p["paragraphs"]:
    paragraph_name = build_node_name(p["section"],p["number"])
    nodes = p["graph"]["nodes"]
    relations = p["graph"]["relations"]
    for node in nodes :
        list_of_cypher_queries += ['CREATE (:concept {name: "'+ node +'"});']
        list_of_cypher_queries += ['MATCH (a:paragraph {name:"' + paragraph_name +'"}), (b:concept {name:"' + node +'"})' + '  CREATE (a)-[:TALKS_ABOUT]->(b);']
    for relation in relations:
        if relation[0] not in nodes:
            list_of_cypher_queries += ['CREATE (:concept {name: "'+ relation[0] +'"});']
            list_of_cypher_queries += ['MATCH (a:paragraph {name:"' + paragraph_name +'"}), (b:concept {name:"' + relation[0] +'"})' + '  CREATE (a)-[:TALKS_ABOUT]->(b);']
        if relation[1] not in nodes:
            list_of_cypher_queries += ['CREATE (:concept {name: "'+ relation[1] +'"});']
            list_of_cypher_queries += ['MATCH (a:paragraph {name:"' + paragraph_name +'"}), (b:concept {name:"' + relation[1] +'"})' + '  CREATE (a)-[:TALKS_ABOUT]->(b);']

        list_of_cypher_queries += ['MATCH (a:concept {name:"' + relation[0] +'"}), (b:concept {name:"' + relation[1] +'"})' + '  CREATE (a)-[:'+ relation[2].upper().replace("-","_").replace(" ","_").replace(".","_") +']->(b);']

In [8]:
with open('cypher_queries.txt', 'w') as f:
    for line in list_of_cypher_queries:
        f.write(f"{line}\n")